In [6]:
import win32com.client
import datetime

In [ ]:
import pandas as pd
pd.read_excel("temp_errores.xlsx")

In [2]:
df_errores = pd.read_excel("temp_errores.xlsx")

In [7]:
#Levantar nuevos errores y hacerle por ejemplo
# df_nuevoserrores = pd.DataFrame('temp_errores.xlsx')
# df_nuevoserrores[0]
lista_n = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]


In [15]:
df_errores[~df_errores[0].isin(lista_n)]

,Unnamed: 0,0,1,2,3
0,0,41,Z:\....Candidatos a borrar\Archivos Viejos\Obs...,"('No se pudo abrir .xlsx', [])",1
1,1,43,Z:\....Candidatos a borrar\Carpetas compartida...,"('No se pudo abrir .xlsx', [])",3
2,2,47,Z:\....Candidatos a borrar\varios\javier zonas...,"('No se pudo abrir .xlsx', [])",7
3,3,48,Z:\....Candidatos a borrar\varios\lista campañ...,"('No se pudo abrir .xlsx', [])",8
4,4,49,Z:\....Candidatos a borrar\varios\lista campañ...,"('No se pudo abrir .xlsx', [])",9
...,...,...,...,...,...
15508,15508,17186,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"(com_error(-2147352567, 'Ocurrió una excepción...",17146
15509,15509,17187,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"(com_error(-2147352567, 'Ocurrió una excepción...",17147
15510,15510,17188,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"(com_error(-2147352567, 'Ocurrió una excepción...",17148
15511,15511,17189,Z:\Desarrollo_Ventas\HOJA DE RUTA FERREYRA\IVA...,"(com_error(-2147352567, 'Ocurrió una excepción...",17149


In [1]:
def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [4]:
import pyodbc
import pandas as pd

In [68]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel with(nolock) where end_processing_date is null'

df = pd.read_sql(query,cnxn)

33021

In [70]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel with(nolock) where end_processing_date is null'

df = pd.read_sql(query,cnxn)
cnxn.close()

#Vamos a levantar los errores anteriores y vamos a relevar los excels_id que no estan en la lista de errores.

errores = pd.read_excel('full_errores.xlsx')

#Buscamos los excel_id que no se encuentran en errores
notIsin = df[~df['excel_id'].isin(errores[0])].reset_index(drop=True)

In [71]:
len(df),len(errores),len(notIsin)

(33021, 15513, 17508)

In [19]:
import datetime

In [20]:
def check_excel(file,excel_id,index,start_processing_date):
    global output
    
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.EnableEvents = False
        print('ENTRO')
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)
            print('ABRIO')

        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)
                    try:
                        for o in x.Ranges(1).QueryTable.Parameters:

                            end_processing_date = datetime.datetime.today()
                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          o.SourceRange.Address,
                                          o.SourceRange.Parent.Name,
                                          o.SourceRange.Value,
                                          excel_id,
                                          index,
                                          start_processing_date,
                                          end_processing_date
                                         ])
                        flag = 1
                    except:
                        flag = 0
                    if flag == 0:

                        try:
                            if len(i.ListObject.QueryTable.Parameters) != 0:

                                for p in i.ListObject.QueryTable.Parameters:
                                    
                                    end_processing_date = datetime.datetime.today()
                                    datos.append([x,
                                                  x.ODBCConnection.CommandText,
                                                  x.ODBCConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value,
                                                  excel_id,
                                                  index,
                                                  start_processing_date,
                                                  end_processing_date
                                                 ])
                            else:
                                end_processing_date = datetime.datetime.today()
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name, #EDIT None
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index,
                                              start_processing_date,
                                              end_processing_date
                                             ])

                        except Exception as e:
                            try:
                                end_processing_date = datetime.datetime.today()
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index,
                                              start_processing_date,
                                              end_processing_date
                                             ])
                            except:
                                try:
                                    if len(i.ListObject.QueryTable.Parameters) != 0:

                                        for p in i.ListObject.QueryTable.Parameters:
                                            end_processing_date = datetime.datetime.today()
                                            datos.append([x,
                                                          x.OLEDBConnection.CommandText,
                                                          x.OLEDBConnection.Connection,
                                                          i.Parent.Name,
                                                          celda,
                                                          coordenada,
                                                          p.SourceRange.Address,
                                                          p.SourceRange.Parent.Name,
                                                          p.SourceRange.Value,
                                                          excel_id,
                                                          index,
                                                          start_processing_date,
                                                          end_processing_date
                                                         ])
                                    else:
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                      x.ODBCConnection.CommandText,
                                                      x.ODBCConnection.Connection,
                                                      i.Parent.Name, #EDIT None
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index,
                                                      start_processing_date,
                                                      end_processing_date
                                                     ])

                                except:
                                    try:
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                      x.OLEDBConnection.CommandText,
                                                      x.OLEDBConnection.Connection,
                                                      i.Parent.Name,
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index,
                                                      start_processing_date,
                                                      end_processing_date
                                                     ])
                                    except:      
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                  None,
                                                  None,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  None,
                                                  None,
                                                  None,
                                                  excel_id,
                                                  index,
                                                  start_processing_date,
                                                  end_processing_date
                                                     ])

            

        except Exception as c:
            return c , []

        end_processing_date = datetime.datetime.today()#FALTA
        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                           excel_id,
                           index,
                           start_processing_date,
                           end_processing_date])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
    
        if type(datos[0]) == str:
            
            output +=[datos]
        else:
            output += datos

        
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0
    except Exception as G:
        print(G)
        return 'No se pudo abrir .xlsx',[]

('No se pudo abrir .xlsx', [])

In [46]:
for index,row in df.iterrows():
    if index == 50:
        break

In [58]:
file = df.loc[17499]['path']+df.loc[17499]['filename']

In [67]:
result = check_excel(archivo,df.loc[17499]['excel_id'],index,'ahora')
result
#('No se pudo abrir .xlsx', [])

(pywintypes.com_error(-2147352567,
                      'Ocurrió una excepción.',
                      (0,
                       'Microsoft Office Excel',
                       "No se puede obtener acceso al archivo 'Z:\\LogisticaDelis\\Ruteos\\Presupuesto\\Presupuesto 1119.xlsx'. Puede haber varios motivos:\n\n• El nombre del archivo o la ruta no existen.\n• Otro programa está usando el archivo.\n• El libro que está intentando guardar tiene el mismo nombre que otro libro que está abierto en estos momentos.",
                       'C:\\Program Files (x86)\\Microsoft Office\\Office12\\3082\\XLMAIN11.CHM',
                       0,
                       -2146827284),
                      None),
 [])

In [57]:
print(archivo)

Z:\LogisticaDelis\Ruteos\Presupuesto\Presupuesto 1119.xlsx


In [9]:
import win32com.client

xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
xl.EnableEvents = False
    #wb = xl.Workbooks.Open(file, None,True)
    #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
    #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
#wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
try:
    wb = xl.Workbooks.Open("Z:\\Desarrollo_Ventas\\HOJA DE RUTA FERREYRA\\IVAN NAONE\\TALLER - IVAN NAONE 26-1.xlsx", None, True)
except Exception as e:
    print(e)

In [13]:
for x in wb.Connections:
    for i in x.Ranges:
        try:
            #PRIMER DATO QUE TRAE LA PRIMER CELDA
            celda = i.Cells(1,1)
            #LA COORDENADA DE LA PRIMER CELDA
            coordenada = i.Cells(1,1).Address
        except Exception as p:
            print(p)
        

In [ ]:
Z:\\Desarrollo_Ventas\\HOJA DE RUTA FERREYRA\\IVAN NAONE\\TALLER - IVAN NAONE 26-1.xlsx

In [7]:

def check_excel(file,excel_id,index,start_processing_date):
    global output
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.EnableEvents = False
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)

        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)
                    try:
                        for o in x.Ranges(1).QueryTable.Parameters:

                            end_processing_date = datetime.datetime.today()
                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          o.SourceRange.Address,
                                          o.SourceRange.Parent.Name,
                                          o.SourceRange.Value,
                                          excel_id,
                                          index,
                                          start_processing_date,
                                          end_processing_date
                                         ])
                        flag = 1
                    except:
                        flag = 0
                    if flag == 0:

                        try:
                            if len(i.ListObject.QueryTable.Parameters) != 0:

                                for p in i.ListObject.QueryTable.Parameters:
                                    
                                    end_processing_date = datetime.datetime.today()
                                    datos.append([x,
                                                  x.ODBCConnection.CommandText,
                                                  x.ODBCConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value,
                                                  excel_id,
                                                  index,
                                                  start_processing_date,
                                                  end_processing_date
                                                 ])
                            else:
                                end_processing_date = datetime.datetime.today()
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name, #EDIT None
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index,
                                              start_processing_date,
                                              end_processing_date
                                             ])

                        except Exception as e:
                            try:
                                end_processing_date = datetime.datetime.today()
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index,
                                              start_processing_date,
                                              end_processing_date
                                             ])
                            except:
                                try:
                                    if len(i.ListObject.QueryTable.Parameters) != 0:

                                        for p in i.ListObject.QueryTable.Parameters:
                                            end_processing_date = datetime.datetime.today()
                                            datos.append([x,
                                                          x.OLEDBConnection.CommandText,
                                                          x.OLEDBConnection.Connection,
                                                          i.Parent.Name,
                                                          celda,
                                                          coordenada,
                                                          p.SourceRange.Address,
                                                          p.SourceRange.Parent.Name,
                                                          p.SourceRange.Value,
                                                          excel_id,
                                                          index,
                                                          start_processing_date,
                                                          end_processing_date
                                                         ])
                                    else:
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                      x.ODBCConnection.CommandText,
                                                      x.ODBCConnection.Connection,
                                                      i.Parent.Name, #EDIT None
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index,
                                                      start_processing_date,
                                                      end_processing_date
                                                     ])

                                except:
                                    try:
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                      x.OLEDBConnection.CommandText,
                                                      x.OLEDBConnection.Connection,
                                                      i.Parent.Name,
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index,
                                                      start_processing_date,
                                                      end_processing_date
                                                     ])
                                    except:      
                                        end_processing_date = datetime.datetime.today()
                                        datos.append([x,
                                                  None,
                                                  None,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  None,
                                                  None,
                                                  None,
                                                  excel_id,
                                                  index,
                                                  start_processing_date,
                                                  end_processing_date
                                                     ])

            

        except Exception as c:
            return c , []

        end_processing_date = datetime.datetime.today()
        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                           excel_id,
                           index,
                           start_processing_date,
                           end_processing_date])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
    
        if type(datos[0]) == str:
            
            output +=[datos]
        else:
            output += datos

        
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0
    except Exception as G:
        return G,[]
output = []

a = check_excel("Z:\\Desarrollo_Ventas\\HOJA DE RUTA FERREYRA\\IVAN NAONE\\TALLER - IVAN NAONE 26-1.xlsx",1,1,"03-03-03")